In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
data = []
labels = []
classes = 43
cur_path = os.getcwd()
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)
    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
            #sim = Image.fromarray(image)
            data.append(image)
            labels.append(i)
        except:
            print("Error loading image")
#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)
print(data.shape, labels.shape)

(39209, 30, 30, 3) (39209,)


In [3]:
#Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size= 0.2, random_state=42)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

y_train = to_categorical(y_train, num_classes = 43)
y_val = to_categorical(y_val, num_classes = 43)

(31367, 30, 30, 3) (7842, 30, 30, 3) (31367,) (7842,)


In [4]:
#Build a CNN model
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation ='relu', input_shape = X_train.shape[1:]))
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.25))
model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.25))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(rate =0.5))
model.add(Dense(43, activation = 'softmax'))

#Compilation of the model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [5]:
#Train and validate the model
history = model.fit(X_train, y_train, batch_size = 64, epochs = 15, validation_data = (X_val, y_val))

Epoch 1/15
491/491 [==============================] - 7s 6ms/step - loss: 3.9151 - accuracy: 0.0653 - val_loss: 3.1394 - val_accuracy: 0.1931
Epoch 2/15
491/491 [==============================] - 2s 5ms/step - loss: 2.0891 - accuracy: 0.4211 - val_loss: 1.0396 - val_accuracy: 0.7286
Epoch 3/15
491/491 [==============================] - 2s 5ms/step - loss: 1.1542 - accuracy: 0.6616 - val_loss: 0.5707 - val_accuracy: 0.8454
Epoch 4/15
491/491 [==============================] - 2s 5ms/step - loss: 0.8520 - accuracy: 0.7450 - val_loss: 0.4266 - val_accuracy: 0.8806
Epoch 5/15
491/491 [==============================] - 2s 5ms/step - loss: 0.6928 - accuracy: 0.7941 - val_loss: 0.4146 - val_accuracy: 0.8860
Epoch 6/15
491/491 [==============================] - 3s 5ms/step - loss: 0.5819 - accuracy: 0.8270 - val_loss: 0.2728 - val_accuracy: 0.9287
Epoch 7/15
491/491 [==============================] - 2s 5ms/step - loss: 0.5435 - accuracy: 0.8377 - val_loss: 0.2152 - val_accuracy: 0.9435
Epoch 

In [4]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score
from PIL import Image
import pandas as pd
import numpy as np

y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
imgs = y_test["Path"].values
data=[]
for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))
X_test=np.array(data)
predict_x=model.predict(X_test) 
pred = np.argmax(predict_x,axis=1)
#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

395/395 [==============================] - 3s 1ms/step
0.9355502771179731


In [12]:
#save model
model.save('model.h5')

In [9]:
#test with real images
from tensorflow import keras
from PIL import Image

model = keras.models.load_model('model.h5')
data = []

#read images
path = "Test/00001.png"
image = Image.open(path)
image = image.resize((30,30))
data.append(np.array(image))

X_test = np.array(data)
predict_x=model.predict(X_test) 
pred = np.argmax(predict_x,axis=1)
print(pred[0])

1/1 [==============================] - 0s 46ms/step
1


In [9]:
#Inference with onnx model
import onnx
onnx_model = onnx.load("traffic_sign.onnx")
onnx.checker.check_model(onnx_model)

In [35]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import torch
from torchvision import transforms

#read images
path = "Test/00001.png"
image = Image.open(path)
image = image.resize((30,30))
convert_tensor = transforms.ToTensor()
img = convert_tensor(image)
print(img)
x = img[0]
ort_sess = ort.InferenceSession('traffic_sign.onnx', providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])
outputs = ort_sess.run(None, {'input': x.numpy()})

# Print Result 
predicted, actual = classes[outputs[0][0].argmax(0)], classes[y]
print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([[[0.2431, 0.3490, 0.3686,  ..., 0.3059, 0.2824, 0.2902],
         [0.2510, 0.3804, 0.3804,  ..., 0.2941, 0.3647, 0.3451],
         [0.2314, 0.3569, 0.3725,  ..., 0.2667, 0.2980, 0.3059],
         ...,
         [0.2039, 0.3176, 0.3137,  ..., 0.2745, 0.3255, 0.3059],
         [0.2078, 0.3137, 0.3059,  ..., 0.2314, 0.2627, 0.2824],
         [0.2118, 0.3137, 0.3059,  ..., 0.2431, 0.2824, 0.2941]],

        [[0.2824, 0.3176, 0.3216,  ..., 0.2510, 0.2549, 0.2667],
         [0.2824, 0.3255, 0.3176,  ..., 0.2471, 0.2706, 0.2667],
         [0.2784, 0.3255, 0.3176,  ..., 0.2471, 0.2627, 0.2706],
         ...,
         [0.2000, 0.2706, 0.2627,  ..., 0.2471, 0.2627, 0.2588],
         [0.1961, 0.2627, 0.2549,  ..., 0.2314, 0.2549, 0.2588],
         [0.1922, 0.2588, 0.2588,  ..., 0.2275, 0.2588, 0.2745]],

        [[0.2392, 0.2471, 0.2549,  ..., 0.2667, 0.2627, 0.2667],
         [0.2353, 0.2549, 0.2471,  ..., 0.2549, 0.2784, 0.2667],
         [0.2353, 0.2588, 0.2471,  ..., 0.2471, 0.2667, 0.

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Invalid Feed Input Name:input

In [7]:
import json
import sys
import os
import time
import numpy as np
import cv2
import onnx
import onnxruntime
from onnx import numpy_helper

model = "traffic_sign.onnx"
path = "Test/00064.png"

def Predict(path = path, model = model):
    #Preprocess the image
    image = cv2.imread(path)
    img = np.dot(image[...,:3], [0.299, 0.587, 0.114])
    img = cv2.resize(img, dsize=(30, 30), interpolation=cv2.INTER_AREA)
    img.resize((64, 30, 30, 3))
    
    data = json.dumps({'data': img.tolist()})
    data = np.array(json.loads(data)['data']).astype('float32')

    session = onnxruntime.InferenceSession(model, None, providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    print(input_name)
    print(output_name)
    
    result = session.run([output_name], {input_name: data})
    print(result)
    prediction=int(np.argmax(np.array(result[0][1]).squeeze(), axis=0))
    print(prediction)

In [8]:
Predict(path = "Test/00064.png")
Predict(path = "Test/00062.png")

conv2d_input
dense_1
[array([[6.5278838e-10, 1.8887455e-03, 7.5699150e-05, ..., 8.2840561e-05,
        1.0507074e-04, 2.7158969e-06],
       [1.7727014e-07, 2.1392223e-01, 8.3310455e-03, ..., 1.9076566e-03,
        7.6611835e-04, 1.6631815e-05],
       [1.7727014e-07, 2.1392223e-01, 8.3310455e-03, ..., 1.9076566e-03,
        7.6611835e-04, 1.6631815e-05],
       ...,
       [1.7727014e-07, 2.1392223e-01, 8.3310455e-03, ..., 1.9076566e-03,
        7.6611835e-04, 1.6631815e-05],
       [1.7727014e-07, 2.1392223e-01, 8.3310455e-03, ..., 1.9076566e-03,
        7.6611835e-04, 1.6631815e-05],
       [1.7727014e-07, 2.1392223e-01, 8.3310455e-03, ..., 1.9076566e-03,
        7.6611835e-04, 1.6631815e-05]], dtype=float32)]
1
conv2d_input
dense_1
[array([[5.9890309e-10, 1.1146225e-03, 4.5248889e-05, ..., 1.3538054e-04,
        1.9246378e-05, 3.0916749e-06],
       [1.7727012e-07, 2.1392222e-01, 8.3310446e-03, ..., 1.9076582e-03,
        7.6611835e-04, 1.6631813e-05],
       [1.7727012e-07, 2.1392